# Ramen training

In [ ]:
#| default_exp 70-ramen-ep-for-wikiseealso-with-random-walk-1-0

In [ ]:
%load_ext autoreload
%autoreload 2

from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
import os,torch, torch.multiprocessing as mp, pickle
from xcai.basics import *
from xcai.graph.random_walk import *
from xcai.models.PPP0XX import DBT021

comet_ml is installed but `COMET_API_KEY` is not set.


In [ ]:
os.environ['WANDB_MODE'] = 'disabled'

In [ ]:
#| export
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
os.environ['WANDB_PROJECT']='xc-nlg_xc-nlg_66-radga-dr-ep-for-wikiseealso'

In [ ]:
data_dir = '/home/scai/phd/aiz218323/Projects/XC/data/'

block = XCBlock.from_cfg(data_dir, 'data_meta', tfm='rm', tokenizer='distilbert-base-uncased', 
                         smp_features=[('lbl2data|cat2lbl2data', 1, 1), ('cat2data', 1, 1)])

/home/scai/phd/aiz218323/.local/lib/python3.9/site-packages/xclib-0.97-py3.9-linux-x86_64.egg/xclib/data/data_utils.py:263: UserWarning: Header mis-match from inferred shape!
  warnings.warn("Header mis-match from inferred shape!")


In [ ]:
data_dir = '/home/scai/phd/aiz218323/Projects/XC/data/'
block = XCBlock.from_cfg(data_dir, 'data', tfm='rm', tokenizer='distilbert-base-uncased', smp_features=[('lbl2data', 1, 1)])

In [ ]:
#| export
pkl_dir = '/home/scai/phd/aiz218323/scratch/datasets/processed'
pkl_file = f'{pkl_dir}/wikiseealso_data-meta_distilbert-base-uncased_rm_ramen-corel.pkl'

In [ ]:
pkl_dir = '/home/scai/phd/aiz218323/scratch/datasets'
pkl_file = f'{pkl_dir}/processed/wikiseealso_data_distilbert-base-uncased_rm_ngame.pkl'

pkl_dir = '/home/scai/phd/aiz218323/scratch/datasets/processed'
pkl_file = f'{pkl_dir}/wikiseealso_data-meta_distilbert-base-uncased_rm_ramen-cat.pkl'

In [ ]:
#| export
with open(pkl_file, 'rb') as file: block = pickle.load(file)

## Random walk

In [ ]:
data_corel = WalkItOff(block.train.dset.data.data_lbl, head_thresh=500, walk_len=400, p_reset=0.8, 
                       topk=10, batch_size=1024, max_dist=1)

Head labels: 82
UTILS::TRIMMING::#ROWS(82)


  0%|          | 0/693082 [00:00<?, ?it/s]

In [ ]:
lbl_corel = WalkItOff(block.train.dset.data.data_lbl.T, head_thresh=500, walk_len=400, p_reset=0.8, 
                      topk=10, batch_size=1024, max_dist=1)

Head labels: 0
UTILS::TRIMMING::#ROWS(0)


  0%|          | 0/312330 [00:00<?, ?it/s]

In [ ]:
with open(f'{pkl_dir}/processed/corelations.pkl', 'wb') as file: 
    pickle.dump((data_corel, lbl_corel), file)

In [ ]:
from xcai.data import MetaXCDataset, XCDataset
from scipy import sparse

In [ ]:
dco_meta = MetaXCDataset('dco', data_corel, sparse.csr_matrix((block.n_lbl, block.train.dset.n_data)), 
                         block.train.dset.data.data_info)

In [ ]:
lco_meta = MetaXCDataset('lco', sparse.csr_matrix((block.train.dset.n_data, block.n_lbl)), lbl_corel, 
                         block.train.dset.data.lbl_info)

In [ ]:
train_dset = XCDataset(block.train.dset.data, dco_meta=dco_meta, lco_meta=lco_meta)
block.train.dset = train_dset

In [ ]:
smp_features=[('lbl2data|lco2lbl2data',1,1), ('dco2data', 1, 1)]
block.collator.tfms.tfms[0].smp_features = smp_features

In [ ]:
b = block.train.one_batch(10)

In [ ]:
b.keys()

dict_keys(['plbl2data_idx', 'plbl2data_data2ptr', 'lbl2data_idx', 'lbl2data_identifier', 'lbl2data_input_text', 'lbl2data_input_ids', 'lbl2data_attention_mask', 'lbl2data_data2ptr', 'plco2lbl_idx', 'plco2lbl_lbl2data2ptr', 'plco2lbl_data2ptr', 'lco2lbl_idx', 'lco2lbl_identifier', 'lco2lbl_input_text', 'lco2lbl_input_ids', 'lco2lbl_attention_mask', 'lco2lbl_lbl2data2ptr', 'lco2lbl_data2ptr', 'pdco2data_idx', 'pdco2data_data2ptr', 'dco2data_idx', 'dco2data_identifier', 'dco2data_input_text', 'dco2data_input_ids', 'dco2data_attention_mask', 'dco2data_data2ptr', 'data_identifier', 'data_input_text', 'data_input_ids', 'data_attention_mask', 'dco2lbl2data_idx', 'dco2lbl2data_identifier', 'dco2lbl2data_input_text', 'dco2lbl2data_input_ids', 'dco2lbl2data_attention_mask', 'dco2lbl2data_data2ptr', 'dco2lbl2data_plbl2data2ptr', 'lco2data_idx', 'lco2data_identifier', 'lco2data_input_text', 'lco2data_input_ids', 'lco2data_attention_mask', 'lco2data_data2ptr'])

In [ ]:
pkl_dir = '/home/scai/phd/aiz218323/scratch/datasets/processed'
pkl_file = f'{pkl_dir}/wikiseealso_data-meta_distilbert-base-uncased_rm_ramen-corel.pkl'

In [ ]:
with open(pkl_file, 'wb') as file: pickle.dump(block, file)

## Training

In [ ]:
#| export
args = XCLearningArguments(
    output_dir='/home/scai/phd/aiz218323/scratch/outputs/70-ramen-ep-for-wikiseealso-with-random-walk-1-0',
    logging_first_step=True,
    per_device_train_batch_size=800,
    per_device_eval_batch_size=800,
    representation_num_beams=200,
    representation_accumulation_steps=100,
    save_strategy="steps",
    evaluation_strategy="steps",
    eval_steps=3000,
    save_steps=3000,
    save_total_limit=5,
    num_train_epochs=300,
    predict_with_representation=True,
    adam_epsilon=1e-6,
    warmup_steps=100,
    weight_decay=0.01,
    learning_rate=2e-4,
    generation_num_beams=10,
    generation_length_penalty=1.5,
    predict_with_generation=True,
    representation_search_type='INDEX',
    group_by_cluster=True,
    num_clustering_warmup_epochs=10,
    num_cluster_update_epochs=5,
    num_cluster_size_update_epochs=25,
    clustering_type='EXPO',
    minimum_cluster_size=2,
    maximum_cluster_size=1600,
    output_concatenation_weight=1.0,
    metric_for_best_model='P@1',
    load_best_model_at_end=True,
    target_indices_key='plbl2data_idx',
    target_pointer_key='plbl2data_data2ptr',
    use_encoder_parallel=True,
    max_grad_norm=None,
    fp16=True,
    label_names=['dco2data_idx', 'dco2data_input_ids', 'dco2data_attention_mask'],
    
    prune_metadata=False,
    num_metadata_prune_epochs=5,
    metadata_prune_batch_size=2048,
    num_metadata_prune_warmup_epochs=0,
    prune_metadata_names=['cat_meta']
)

In [ ]:
#| export
metric = PrecRecl(block.n_lbl, block.test.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200])

In [ ]:
#| export
bsz = max(args.per_device_train_batch_size, args.per_device_eval_batch_size)*torch.cuda.device_count()

model = DBT021.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', bsz=bsz, tn_targ=5000, margin=0.3, tau=0.1, 
                               apply_softmax=True, n_negatives=10, m_lw=0.15, meta_prefix='dco', use_encoder_parallel=True, 
                               task_repr_type='pool', meta_repr_type='pool')
model.init_dr_head()

Some weights of DBT021 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['encoder.dr_layer_norm.bias', 'encoder.dr_layer_norm.weight', 'encoder.dr_projector.bias', 'encoder.dr_projector.weight', 'encoder.dr_transform.bias', 'encoder.dr_transform.weight', 'encoder.meta_layer_norm.bias', 'encoder.meta_layer_norm.weight', 'encoder.meta_projector.bias', 'encoder.meta_projector.weight', 'encoder.meta_transform.bias', 'encoder.meta_transform.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#| export
learn = XCLearner(
    model=model, 
    args=args,
    train_dataset=block.train.dset,
    eval_dataset=block.test.dset,
    data_collator=block.collator,
    compute_metrics=metric,
)

/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
learn.train()

In [ ]:
#| export
if __name__ == '__main__':
    mp.freeze_support()
    learn.train()

In [ ]:
o = learn.predict(learn.eval_dataset)

  0%|          | 0/196 [00:00<?, ?it/s]

/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/site-packages/scipy/sparse/_index.py:145: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
